In [28]:
import pyodbc
import pandas as pd
import requests
from datetime import datetime

# Calculando Demanda

Vamos pegar um conjunto de dados de previsão para as regiões de saúde de Goiás

In [29]:
df = pd.read_excel("previsoes_go.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Estado    649 non-null    object        
 1   codibge   649 non-null    int64         
 2   mae_mlp   649 non-null    float64       
 3   mape_mlp  649 non-null    float64       
 4   mes_ano   649 non-null    object        
 5   qtd       649 non-null    float64       
 6   data      649 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 35.6+ KB


In [30]:
df = df.drop(columns=['Estado','mae_mlp','mape_mlp','mes_ano'])
df.head()

,codibge,qtd,data
0,52001,2228.074325,2022-12-01
1,52002,1105.240907,2023-01-01
2,52011,637.147580,2023-01-01
3,52004,1044.177786,2023-01-01
4,52013,176.677516,2023-01-01


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   codibge  649 non-null    int64         
 1   qtd      649 non-null    float64       
 2   data     649 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 15.3 KB


In [32]:
df['qtd'] = df['qtd'].round()
df['qtd'] = df['qtd'].astype(int)

df['ano_mes']=df['data'].astype(str)
df['ano_mes'] = df['ano_mes'].str.slice(start = 0, stop = 7)

df.head()

,codibge,qtd,data,ano_mes
0,52001,2228,2022-12-01,2022-12
1,52002,1105,2023-01-01,2023-01
2,52011,637,2023-01-01,2023-01
3,52004,1044,2023-01-01,2023-01
4,52013,177,2023-01-01,2023-01


Vamos utilizar apenas a **população SUS dependente**. Portanto, vamos deduzir um percentual da população detentora de plano de saúde. 

In [33]:
plano = pd.read_csv("beneficiarios_plano_saude.csv")

plano = plano.loc[(plano.mes == 6)]

plano = plano[['cod_regsaud','percentual_pop_coberta']]

df = df.merge(plano, left_on='codibge', right_on='cod_regsaud', how = 'left')

df['qtd_sus'] = df['qtd'] * ((100 - df['percentual_pop_coberta'])/100)
df['qtd_sus'] = df['qtd_sus'].round()
df['qtd_sus'] = df['qtd_sus'].astype(int)


df.head()

,codibge,qtd,data,ano_mes,cod_regsaud,percentual_pop_coberta,qtd_sus
0,52001,2228,2022-12-01,2022-12,52001,48.75,1142
1,52002,1105,2023-01-01,2023-01,52002,24.64,833
2,52011,637,2023-01-01,2023-01,52011,27.56,461
3,52004,1044,2023-01-01,2023-01,52004,13.86,899
4,52013,177,2023-01-01,2023-01,52013,14.49,151


Vamos usar o API para verificar os procedimentos necessários para cada nascido vivo de um mês de referência e região de saúde

In [34]:
dfs = [] 

for _, row in df.iterrows():
    qtd_nascidos = round(row['qtd_sus'])
    ano_mes = row['ano_mes']
    url = f'http://200.137.215.27:5025/calcula_procedimentos?mes_ano={ano_mes}&nascidos_vivos={qtd_nascidos}'
    print("Chamando", url)
    resultado = requests.get(url)
    df_it = pd.DataFrame(resultado.json())
    df_it['ano_mes'] = row['ano_mes'] 
    df_it['qtd_nascidos'] = qtd_nascidos
    df_it['regiao'] = row['codibge']
    dfs.append(df_it)
    df_full = pd.concat(dfs, ignore_index=True)

Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2022-12&nascidos_vivos=1142
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=833
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=461
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=899
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=151
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=170
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=101
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=339
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=186
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-01&nascidos_vivos=144
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=202

Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-05&nascidos_vivos=222
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-05&nascidos_vivos=1245
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=870
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=473
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=984
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=159
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=167
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=115
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=341
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-06&nascidos_vivos=186
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=202

Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-10&nascidos_vivos=106
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-10&nascidos_vivos=220
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-10&nascidos_vivos=1144
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=781
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=459
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=867
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=152
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=155
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=103
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=2023-11&nascidos_vivos=329
Chamando http://200.137.215.27:5025/calcula_procedimentos?mes_ano=202

ConnectionError: HTTPConnectionPool(host='200.137.215.27', port=5025): Max retries exceeded with url: /calcula_procedimentos?mes_ano=2024-03&nascidos_vivos=148 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025338E770A0>: Failed to establish a new connection: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu'))

In [1]:
df_full.head()
#df_full.to_csv("arquivos PBI/total_procedimentos_PBI.csv")

NameError: name 'df_full' is not defined

**Descrição de variáveis** 
- codigo_sigtap: código de identificação única de um procedimento (um procedimento pode ocorrer múltiplas vezes ao longo da gestação, como o pré-natal)
- mes: mês de referência que aquele procedimento deve ser realizado (mês 1 = primeiro mês de gestação)
- parametro: quantos procedimentos naquele mês para o total de gestantes devem ser realizados (ex.: existem procedimentos que só metade das gestantes precisam fazer) 
- procedimento: nome do procedimento
- tipo_procedimento: tipo de procedimento
- Público: Pode ser gestantes (risco habiual ou alto risco) ou nascidos. Via de regra pode ser todas ou, para alguns procedimentos, as de alto risco
- nivel_atencao: nível de atenção onde aquele procedimento geralmente é realizado
- quantidade: quantidade de procedimentos que devem ser realizados tendo em vista as colunas parametro e qtd_nascidos
- mês_procedimento_realizado: mês da gestação que aquele procedimento deve ser realizado, tomando como referência as colunas ano_mes e mes
- qtd_nascidos: quantidade de registros de nascidos vivos para a região de saúde, já descontado o S
- regiao: código IBGE que identifica a região de saúde

# Calculando procedimentos por mês 

Agora vamos calcular a quantidade de procedimentos por mês de realização do procedimento

In [36]:
procedimentos_mes = df_full.groupby(['regiao','ano_mes','mês_procedimento_realizado','nivel_atencao',
                                     'tipo_procedimento','codigo_sigtap','qtd_nascidos'])['quantidade'].sum().reset_index()

In [37]:
procedimentos_mes[["mês_procedimento_realizado"]] = procedimentos_mes[["mês_procedimento_realizado"]].apply(pd.to_datetime) 
procedimentos_mes['codigo_sigtap'] = procedimentos_mes['codigo_sigtap'].astype('str') 

procedimentos_mes.head()

,regiao,ano_mes,mês_procedimento_realizado,nivel_atencao,tipo_procedimento,codigo_sigtap,qtd_nascidos,quantidade
0,52001,2022-12,2022-04-01,APS,Consultas ou Visitas,301010110,1142,1142.0
1,52001,2022-12,2022-05-01,APS,Consultas ou Visitas,301010110,1142,1142.0
2,52001,2022-12,2022-05-01,APS,Consultas ou Visitas,301010137,1142,1142.0
3,52001,2022-12,2022-05-01,APS,Exames,202020355,1142,1142.0
4,52001,2022-12,2022-05-01,APS,Exames,202020380,1142,1142.0


Considerando o total de procedimentos, vamos computar em horas primeiro. Para isso, vamos ler uma planilha com estimativas de tempos dos procedimento. **Em uma implementação de sistema, é requisito que os tempos sejam ajustados pelo usuário.** 

In [38]:
tempos = pd.read_excel("calendario-procedimentos.xlsx", sheet_name="procedimentos_tempo") 
tempos['codigo_sigtap'] = tempos['codigo_sigtap'].astype('str')
tempos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   codigo_sigtap  61 non-null     object 
 1   procedimento   61 non-null     object 
 2   Tempo          61 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.6+ KB


In [39]:
df_tempos = procedimentos_mes.merge(tempos, on = 'codigo_sigtap', how = 'left')

df_tempos['horasProfissionais'] = df_tempos['quantidade'] * df_tempos['Tempo'] 

df_tempos.head()

,regiao,ano_mes,mês_procedimento_realizado,nivel_atencao,tipo_procedimento,codigo_sigtap,qtd_nascidos,quantidade,procedimento,Tempo,horasProfissionais
0,52001,2022-12,2022-04-01,APS,Consultas ou Visitas,301010110,1142,1142.0,Consulta pré-natal,0.500000,571.000000
1,52001,2022-12,2022-05-01,APS,Consultas ou Visitas,301010110,1142,1142.0,Consulta pré-natal,0.500000,571.000000
2,52001,2022-12,2022-05-01,APS,Consultas ou Visitas,301010137,1142,1142.0,Visita domiciliar,0.500000,571.000000
3,52001,2022-12,2022-05-01,APS,Exames,202020355,1142,1142.0,Eletroforese de Hemoglobina,0.166667,190.333333
4,52001,2022-12,2022-05-01,APS,Exames,202020380,1142,1142.0,Hemograma,0.083333,95.166667


Agora vamos ver os perfis de profissionais que atuam nos procedimentos. Temos profissionais que podem atuar em mais de um procedimento (ex.: médicos e enfermeiros podem atuar no pré-natal). Por isso que tem essa contagem resultando no df mix. Se o procedimento é realizado por dois profissionais, dividimos as horasProfissionais para duas categorias. 

In [40]:
df_profissionais = pd.read_excel("calendario-procedimentos.xlsx", sheet_name="procedimentos_profissionais")

df_profissionais['codigo_sigtap'] = df_profissionais['codigo_sigtap'].astype('str')

mix = df_profissionais.groupby(['codigo_sigtap'])['CBO'].count().reset_index()

mix = mix.rename(columns={"CBO":"qtd"})

In [41]:
df_tempos = df_tempos.merge(mix, on = 'codigo_sigtap', how = 'left') 

df_tempos['horas_categoria'] = df_tempos['horasProfissionais'] / df_tempos['qtd']

df_tempos.head()

,regiao,ano_mes,mês_procedimento_realizado,nivel_atencao,tipo_procedimento,codigo_sigtap,qtd_nascidos,quantidade,procedimento,Tempo,horasProfissionais,qtd,horas_categoria
0,52001,2022-12,2022-04-01,APS,Consultas ou Visitas,301010110,1142,1142.0,Consulta pré-natal,0.500000,571.000000,2,285.500000
1,52001,2022-12,2022-05-01,APS,Consultas ou Visitas,301010110,1142,1142.0,Consulta pré-natal,0.500000,571.000000,2,285.500000
2,52001,2022-12,2022-05-01,APS,Consultas ou Visitas,301010137,1142,1142.0,Visita domiciliar,0.500000,571.000000,1,571.000000
3,52001,2022-12,2022-05-01,APS,Exames,202020355,1142,1142.0,Eletroforese de Hemoglobina,0.166667,190.333333,1,190.333333
4,52001,2022-12,2022-05-01,APS,Exames,202020380,1142,1142.0,Hemograma,0.083333,95.166667,1,95.166667


Aqui dividimos o quantitativo de profissionais por 126, que é o correspondente de horas líquidas mensais de um profissional de 40h semanais para calcular a métrica da FTE40

In [42]:
df_tempo_prof = df_tempos.merge(df_profissionais, on = 'codigo_sigtap', how = 'left')

df_tempo_prof = df_tempo_prof.groupby(['regiao', 'mês_procedimento_realizado','nivel_atencao', 'qtd_nascidos', 
                                       'ano_mes','CBO'])['horas_categoria'].sum().reset_index()

df_tempo_prof['fte40'] = df_tempo_prof['horas_categoria']/126

df_tempo_prof.head()

,regiao,mês_procedimento_realizado,nivel_atencao,qtd_nascidos,ano_mes,CBO,horas_categoria,fte40
0,52001,2022-04-01,APS,1142,2022-12,223505,285.5,2.265873
1,52001,2022-04-01,APS,1142,2022-12,225130,285.5,2.265873
2,52001,2022-05-01,APS,1142,2022-12,221205,2226.9,17.673810
3,52001,2022-05-01,APS,1142,2022-12,223505,513.9,4.078571
4,52001,2022-05-01,APS,1142,2022-12,225130,513.9,4.078571


In [43]:
#df_tempo_prof.to_csv("df_prof_go.csv")